# H3 Speed Visualization

This notebook queries the `mdl_speed_h3_hourly` model from BigQuery and visualizes the average vehicle speed on a map using H3 hexagons.

### 1. Setup

In [8]:
import plotly.express as px
import h3
import pandas as pd
from google.cloud import bigquery
import google.auth
import plotly.graph_objects as go


### 2. Query BigQuery Data

In [ ]:
# Load position model
project_id = "regal-dynamo-470908-v9"
dataset_id = "auckland_data_dev"
table_id = "agg_position_h3_day"

# Set up the BigQuery client
client = bigquery.Client(project=project_id)

# Define the query to get the latest data, filtering for non-null h3_index
query = f"""
    WITH latest_date AS (
        SELECT MAX(service_date) as max_date
        FROM `{project_id}.{dataset_id}.{table_id}`
    )
    SELECT
        t.h3_index,
        t.position_count
    FROM `{project_id}.{dataset_id}.{table_id}` t
    CROSS JOIN latest_date
    WHERE t.service_date = latest_date.max_date
      AND t.h3_index IS NOT NULL
      AND t.route_mode IS NOT NULL
    LIMIT 1000
"""

# Execute the query and load to a DataFrame
df = client.query(query).to_dataframe()

# Strip the quotes from the h3_index column
if not df.empty:
    df['h3_index'] = df['h3_index'].str.strip('"')

df_position = df
len(df)

E0000 00:00:1762459474.402804 10132293 alts_credentials.cc:93] ALTS creds ignored. Not running on GCP and untrusted ALTS is not enabled.


13

In [10]:
# Get speed model
project_id = "regal-dynamo-470908-v9"
dataset_id = "auckland_data_dev"
table_id = "mdl_speed_h3_hourly"

# Set up the BigQuery client
client = bigquery.Client(project=project_id)

# Define the query to get the latest data, filtering for non-null h3_index
query = f"""
    WITH latest_date AS (
        SELECT MAX(service_date) as max_date
        FROM `{project_id}.{dataset_id}.{table_id}`
    ),
    latest_hour AS (
        SELECT MAX(hour) as max_hour
        FROM `{project_id}.{dataset_id}.{table_id}`
        WHERE service_date = (SELECT max_date FROM latest_date)
    )
    SELECT
        t.h3_index,
        t.avg_speed_kmh
    FROM `{project_id}.{dataset_id}.{table_id}` t
    CROSS JOIN latest_date
    CROSS JOIN latest_hour
    WHERE t.service_date = latest_date.max_date
      AND t.hour = latest_hour.max_hour
      AND t.h3_index IS NOT NULL
"""

# Execute the query and load to a DataFrame
df = client.query(query).to_dataframe()

# Strip the quotes from the h3_index column
if not df.empty:
    df['h3_index'] = df['h3_index'].str.strip('"')

df_speed = df
len(df)

E0000 00:00:1762459516.697116 10132293 alts_credentials.cc:93] ALTS creds ignored. Not running on GCP and untrusted ALTS is not enabled.


651

In [11]:
import plotly.graph_objects as go

# 1. Define resolutions, center point, and k-ring size for coverage.
child_resolution = 8
parent_resolution = child_resolution - 1 # Resolution 7 for larger hexagons
auckland_center_lat = -36.85
auckland_center_lon = 174.76
k_ring_size = 100

# 2. Generate the primary grid of smaller (child) hexagons.
center_hex = h3.latlng_to_cell(auckland_center_lat, auckland_center_lon, child_resolution)
child_hexagons = h3.grid_disk(center_hex, k_ring_size)

# 3. Find the unique set of larger (parent) hexagons that contain the child hexagons.
parent_hexagons = {h3.cell_to_parent(h, parent_resolution) for h in child_hexagons}

# 4. Helper function to create a line trace for a set of hexagons.
def create_h3_trace(hex_ids, color, width):
    """Generates a Scattermap trace for a set of H3 hexagons."""
    all_lats = []
    all_lons = []
    for hex_id in hex_ids:
        boundary = h3.cell_to_boundary(hex_id)
        lats, lons = zip(*boundary)
        all_lats.extend(list(lats) + [lats[0], None]) # Close polygon and add None
        all_lons.extend(list(lons) + [lons[0], None])
    
    return go.Scattermap(
        mode="lines",
        lon=all_lons,
        lat=all_lats,
        line=dict(width=width, color=color),
        hoverinfo='none'
    )

# 5. Create traces for both parent and child hexagons using lighter shades.
trace_child = create_h3_trace(child_hexagons, '#d9d9d9', 1)
trace_parent = create_h3_trace(parent_hexagons, '#d9d9d9', 2)

### 3. Plots

In [30]:
# --- Plotting Logic for agg_position_h3_day ---
df = df_position
# 1. Create a GeoJSON Feature for each H3 cell
features_position = []
for _, row in df.iterrows():
    try:
        boundary = h3.cell_to_boundary(row["h3_index"])
        geojson_boundary = [[lon, lat] for lat, lon in boundary]
        features_position.append({
            "type": "Feature",
            "geometry": {"type": "Polygon", "coordinates": [geojson_boundary]},
            "id": row["h3_index"]
        })
    except h3.H3ValueError:
        pass

geojson_collection_position = {"type": "FeatureCollection", "features": features_position}

# 2. Create the choropleth map
fig_position = px.choropleth_map(
    df,
    geojson=geojson_collection_position,
    locations="h3_index",
    color="position_count",
    color_continuous_scale="Viridis",
    map_style="carto-positron",
    zoom=10,
    center={"lat": -36.85, "lon": 174.76},
    opacity=0.6,
    title="H3 Unmonitored Position Count (Rail) with Lattice"
)

# 3. Add the lattice traces to the figure
#fig_position.add_trace(trace_parent)
#fig_position.add_trace(trace_child)

fig_position.update_layout(margin={"r":0, "t":40, "l":0, "b":0}, width=1200, height=800)
fig_position.show()

In [29]:
# --- Plotting Logic for mdl_speed_h3_hourly ---
df = df_speed

# 1. Create a GeoJSON Feature for each H3 cell
features_speed = []
for _, row in df.iterrows():
    try:
        boundary = h3.cell_to_boundary(row["h3_index"])
        geojson_boundary = [[lon, lat] for lat, lon in boundary]
        features_speed.append({
            "type": "Feature",
            "geometry": {"type": "Polygon", "coordinates": [geojson_boundary]},
            "id": row["h3_index"]
        })
    except h3.H3ValueError:
        pass

geojson_collection_speed = {"type": "FeatureCollection", "features": features_speed}

# 2. Create the choropleth map
fig_speed = px.choropleth_map(
    df,
    geojson=geojson_collection_speed,
    locations="h3_index",
    color="avg_speed_kmh",
    color_continuous_scale="Viridis", # Using a different color scale
    map_style="carto-positron",
    zoom=9,
    center={"lat": -36.85, "lon": 174.76},
    opacity=0.4,
    title="H3 Average Speed (km/h) with Lattice"
)

# 3. Add the lattice traces to the figure
#fig_speed.add_trace(trace_parent)
#fig_speed.add_trace(trace_child)

fig_speed.update_layout(margin={"r":0, "t":40, "l":0, "b":0}, width=1200, height=800)
fig_speed.show()